# Recon 3D CHO GPRs
## Description
This notebook describes the process by which we can mine information from Recon3D to add it to our reconstruction in CHO cells. First we map all the CHO GPRs corresponding to the Recon3D Human GPRs and a dataset containing all the reactions from Recon3D with their corresponding CHO GPRs is generated (1), then a dataset containing reactions from Recon3D that are not present in our reconstruction, along with annotated CHO GPRs, is generated in order to add it to our reconstruction (2).

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import cobra
from cobra.io.mat import load_matlab_model

import gspread

### 1. Finding CHO orthologs for Human GPRs in Recon3D
Using the dataset generated in the **Orthologs** notebook, we can map all the CHO genes in the recon 3D dataset GPR column and generate a new column called "CHO GPR"

In [2]:
#Generate recon3d df from the recon3d supplementary data
df1 = pd.read_excel('../../Data/GPR_Curation/recon3d_gprs.xlsx')
df1

,m_reaction,m_gene_reaction_rule,m_metabolites,m_subsystem,m_gene,seq_uniprot,seq_len,seq_file,seq_num_structures,seq_num_experimental_structures,struct_id,struct_chain_id,struct_chain_seq_coverage,struct_is_experimental,struct_pdb,struct_resolution,struct_chemicals,struct_file
0,11DOCRTSLte,5243.1,h2o[c];atp[c];h[c];11docrtsl[c];11docrtsl[e];p...,"Transport, extracellular",5243.1,P08183-1,1280.0,P08183-1.fasta,21.0,21.0,3g61-A,A,80.5,1.0,3g61,4.35,2J8,3g61-A_clean.pdb
1,11DOCRTSTRNte,5243.1,atp[c];pi[c];11docrtstrn[c];h[c];11docrtstrn[e...,"Transport, extracellular",5243.1,P08183-1,1280.0,P08183-1.fasta,21.0,21.0,3g61-A,A,80.5,1.0,3g61,4.35,2J8,3g61-A_clean.pdb
2,12DHCHOLabc,8714.3 or 8647.1 or 8714.2 or 8714.1 or 1244.1...,atp[c];12dhchol[e];12dhchol[c];pi[c];h[c];h2o[...,"Transport, extracellular",1244.1,Q92887-1,1545.0,Q92887-1.fasta,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12DHCHOLabc,8714.3 or 8647.1 or 8714.2 or 8714.1 or 1244.1...,atp[c];12dhchol[e];12dhchol[c];pi[c];h[c];h2o[...,"Transport, extracellular",8647.1,O95342-1,1321.0,O95342-1.fasta,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12DHCHOLabc,8714.3 or 8647.1 or 8714.2 or 8714.1 or 1244.1...,atp[c];12dhchol[e];12dhchol[c];pi[c];h[c];h2o[...,"Transport, extracellular",10257.1,O15439-1,1325.0,O15439-1.fasta,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22107,r2525,8501.1,gln_L[c];gln_L[e],"Transport, extracellular",8501.1,O75387-1,559.0,O75387-1.fasta,2.0,0.0,H16666-X,X,100.0,0.0,NaN,NaN,NaN,NP_003618.1_model1_clean-X_clean.pdb
22108,r2526,8501.1,ser_L[e];ser_L[c],"Transport, extracellular",8501.1,O75387-1,559.0,O75387-1.fasta,2.0,0.0,H16666-X,X,100.0,0.0,NaN,NaN,NaN,NP_003618.1_model1_clean-X_clean.pdb
22109,r2532,8501.1,asn_L[c];asn_L[e],"Transport, extracellular",8501.1,O75387-1,559.0,O75387-1.fasta,2.0,0.0,H16666-X,X,100.0,0.0,NaN,NaN,NaN,NP_003618.1_model1_clean-X_clean.pdb
22110,r2534,8501.1,thr_L[c];thr_L[e],"Transport, extracellular",8501.1,O75387-1,559.0,O75387-1.fasta,2.0,0.0,H16666-X,X,100.0,0.0,NaN,NaN,NaN,NP_003618.1_model1_clean-X_clean.pdb


In [4]:
#Generate another df from the recon3d model
recon3d_model = load_matlab_model('../../Data/Reconciliation/models/Recon3D_301.mat')
recon3d_model

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-06


No defined compartments in model Recon3D. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


Name,Recon3D
Memory address,30c4a50f0
Number of metabolites,8399
Number of reactions,13543
Number of genes,3697
Number of groups,111
Objective expression,1.0*biomass_reaction - 1.0*biomass_reaction_reverse_32a6c
Compartments,"c, l, m, r, e, x, n, g, i"


In [5]:
#Generate another df from the recon3d model
attributes = []
for reaction in tqdm(recon3d_model.reactions):
    attributes.append([reaction.id, reaction.name, reaction.reaction, reaction.gpr, 
                       reaction.subsystem, reaction.lower_bound, reaction.upper_bound])

df2 = pd.DataFrame(data=attributes, columns=['m_reaction', 'Reaction Name', 'm_metabolites', 'm_gene_reaction_rule', 'm_subsystem', 'Lower bound', 'Upper bound'])
df2['m_reaction'] = df2['m_reaction'].str.replace("[", "_")
df2['m_reaction'] = df2['m_reaction'].str.replace("]", "")
df2

  0%|          | 0/13543 [00:00<?, ?it/s]

/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_4135/3167771947.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['m_reaction'] = df2['m_reaction'].str.replace("[", "_")
/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_4135/3167771947.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['m_reaction'] = df2['m_reaction'].str.replace("]", "")


,m_reaction,Reaction Name,m_metabolites,m_gene_reaction_rule,m_subsystem,Lower bound,Upper bound
0,10FTHF5GLUtl,"5-Glutamyl-10Fthf Transport, Lysosomal",10fthf5glu[c] --> 10fthf5glu[l],,"Transport, lysosomal",0.0,1000.0
1,10FTHF5GLUtm,"5-Glutamyl-10Fthf Transport, Mitochondrial",10fthf5glu[m] --> 10fthf5glu[c],,"Transport, mitochondrial",0.0,1000.0
2,10FTHF6GLUtl,"6-Glutamyl-10Fthf Transport, Lysosomal",10fthf6glu[c] --> 10fthf6glu[l],,"Transport, lysosomal",0.0,1000.0
3,10FTHF6GLUtm,"6-Glutamyl-10Fthf Transport, Mitochondrial",10fthf6glu[m] --> 10fthf6glu[c],,"Transport, mitochondrial",0.0,1000.0
4,10FTHF7GLUtl,"7-Glutamyl-10Fthf Transport, Lysosomal",10fthf7glu[c] --> 10fthf7glu[l],,"Transport, lysosomal",0.0,1000.0
...,...,...,...,...,...,...,...
13538,CYOR_u10mi,CYOR_u10mi,2.0 ficytC[m] + 2.0 h[m] + q10h2[m] --> 2.0 fo...,(7384.1 and 7388.1 and 4519.1 and 29796.2 and ...,Oxidative phosphorylation,0.0,1000.0
13539,Htmi,Htmi,h[i] --> h[m],9016.1 or 7352.2 or 7352.1 or 7351.1 or 9016.2...,"Transport, mitochondrial",0.0,1000.0
13540,NADH2_u10mi,NADH2_u10mi,5.0 h[m] + nadh[m] + q10[m] --> 4.0 h[i] + nad...,(4715.1 and 4720.1 and 4719.1 and 4700.1 and 5...,Oxidative phosphorylation,0.0,1000.0
13541,CYOOm3i,CYOOm3i,4.0 focytC[m] + 7.92 h[m] + o2[m] --> 4.0 ficy...,1351.1 and 1347.1 and 1329.1 and 1327.1 and 34...,Oxidative phosphorylation,0.0,1000.0


In [ ]:
# Unify df1 and df2 into one dataset containing all the necesary information
recon3d = pd.concat([df2, df1])
recon3d = recon3d.groupby('m_reaction').first()
recon3d = recon3d.reset_index(drop = False)
recon3d['m_reaction'] = recon3d['m_reaction'].str.replace('_hs$', '', regex=True) # Erase the _hs at the end of the reaction IDs
recon3d['m_gene_reaction_rule'] = recon3d['m_gene_reaction_rule'].replace(np.nan,'',regex=True)
recon3d.to_excel('../../Data/Reconciliation/datasets/recon3D_all_reactions.xlsx')

In [ ]:
# Generate orthologs dict from the dataset generated in the Orthologs notebook
orthologs = pd.read_excel('../../Data/GPR_Curation/orthologs.xlsx', dtype=str)
orthologs.fillna('', inplace=True)
orthologs_dict = orthologs.set_index('Human GeneID')['CHO GeneID'].to_dict()
orthologs_dict

In [ ]:
# Extract GPR info from Recon3D and swap gene IDs from human to CHO
import re
cho_gpr = []

for row in recon3d['m_gene_reaction_rule']:
    row = str(row)
    if row != "":
        gpr = re.findall('[\d.]*\d+', row)
        new_gpr = row
        for g in gpr:
            human_g = g.split('.')[0]
            try:
                cho_g = orthologs_dict[human_g]
                if cho_g == '':
                    cho_g = f'h{human_g}'
            except:
                cho_g = f'h{human_g}'
            new_gpr = new_gpr.replace(g, cho_g)
    elif row == '':
        new_gpr = ''
        
    cho_gpr.append(new_gpr)

In [ ]:
cho_gpr

In [ ]:
# Generate Recon3D dataset with the addition of GPR with CHO genes
recon3d['CHO GPR'] = cho_gpr
recon3d.to_excel('../../Data/GPR_Curation/recon3D_chogprs.xlsx')
recon3d

The dataset generated here will then be used in the **Reactions** notebook **Part 6**

### 2. Adittion of new reactions from Recon 3D
Recon3D reactions with mapped CHO GPRs are added to our reconstruction

In [ ]:
# Subset of Recon 3D dataset with information on CHO GPRs
recon3d_cho = recon3d[recon3d['CHO GPR'] != '']
recon3d_cho = recon3d_cho.reset_index(drop=True)

In [ ]:
recon3d_cho['m_metabolites'] = recon3d_cho['m_metabolites'].str.replace("[", "_", regex=True)
recon3d_cho['m_metabolites'] = recon3d_cho['m_metabolites'].str.replace("]", "", regex=True)
recon3d_cho

In [ ]:
#Retreive information from the reactions in our reconstruction in Google Sheets

# give service account details to gspread
sa = gspread.service_account(filename='../credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction')

# we also need to specify the page name before getting the data. In this case we use the Rxns and Metabolites sheet.
rxns_sheet = cho_recon.worksheet('Rxns')

# We can extract the data using the get_all_records method and create pd DataFrames

# Reactions IDs, names, formulas, GPRs
rxns = pd.DataFrame(rxns_sheet.get_all_records())
del rxns[rxns.columns[0]]
rxns

In [ ]:
# List of all the reactions in our reconstruction
cho_rxns_list = list(rxns['Reaction'])

# List of all the reactions with CHO GPRs in Recon3D 
recon3D_rxns_list = list(recon3d_cho['m_reaction'])

In [ ]:
# List of Recon3D reactions that are in our reconstruction
rxns_in_cho = []

# List of Recon3D reactions that are NOT in our reconstruction
rxns_not_in_cho = []

for rxn in recon3D_rxns_list:
    if rxn in cho_rxns_list:
        rxns_in_cho.append(rxn)
    elif rxn not in cho_rxns_list:
        rxns_not_in_cho.append(rxn)
              

In [ ]:
# Generation of a dataset containing information about Recon3D reactions that are not in our reconstruction
subset_df = recon3d_cho[recon3d_cho['m_reaction'].isin(rxns_not_in_cho)]
subset_df

In [ ]:
#This dataset contains all the reactions added from Recon 3D

rxns_recon3d_toadd = subset_df[['m_reaction', 'Reaction Name', 'm_metabolites', 'm_subsystem', 'CHO GPR', 'Lower bound', 'Upper bound']]
rxns_recon3d_toadd.to_excel('../../Data/Reconciliation/datasets/rxns_recon3d_toadd.xlsx')